<a href="https://colab.research.google.com/github/jansoe/KISchule/blob/main/A3_jan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  3. Ein Blick ins Innere der neuronalen Netze

Im letzten Aufgabenblatt haben Sie im Schnelldurchlauf die Geschichte der Bilderkennung mit neuronalen Netzen durchlebt. Sie haben kleinere Netze selbst erzeugt und trainiert und große, vortrainierte mit `Keras` heruntergeladen. 
Doch was passiert in so einem Netzwerk eigentlich? Das einzelne Perzeptron mit zweidimensionalen Inputs kann man noch relativ leicht visualisieren, doch schon bei den kleineren, mehrschichtigen und erst recht bei den riesigen modernen Convolutional-Networks entzieht es sich unserer Vorstellungskraft, welche Rolle einzelne Schichten oder gar einzelne Gewichte spielen.

Diese Woche lernen Sie Methoden kennen, mit denen Sie die Aktivität in neuronalen Netzen analysieren können. Im Normalfall besteht die Anwendung der Netze darin, dass man die Paramter eines Modells so anpasst, dass für einen gewissen Input ein erwünschter Output entsteht. Diese Woche drehen wir das Problem um: Wir lassen die Parameter der Netzwerke unverändert, und machen uns am Input zu schaffen.

Sie lernen, wie man ein Input-Bild so optimiert, dass bestimmte Zellen besonders stark aktiviert werden. So können wir uns bei verschieden großen Netzwerken ansehen, auf welche Merkmale der Eingabedaten die einzelnen Schichten reagieren. Mit diesem Hintergrundwissen können Sie dann nachvollziehen, wie die [Traumbilder](https://de.wikipedia.org/wiki/DeepDream) der tiefen Netze zustande kommen, die vor einiger Zeit durch die [Medien](https://www.zeit.de/digital/internet/2015-07/neuronale-netzwerke-google-inception) gingen.

Im Anschluss sind Sie bereit für Ihren ersten *Hack*: Mit den gleichen Techniken lernen Sie Bilder so zu verändern, dass ein Klassifizierungsnetzwerk diese falsch interpretiert, obwohl für das menschliche Auge kein Unterschied zu korrekt klassifizierten Pendants zu erkennen ist.

Als letztes gehen wir dann nochmal auf den Image-Style-Tranfer ein, den Sie bereits aus der letzten Vorlesung kennen. Sie werden sehen, dass auch hier eine Optimierung des Inputs nach bestimmten Kriterien erfolgt. 

Das Verfahren ist wie gehabt:
- Erstellen Sie eine Kopie dieses Notebooks in ihrem Google Drive (vorgeschlagene Umbenennung: "A3 - Vorname, Nachname")
- Editieren Sie die Text- und Codezellen.
- Teilen Sie einen Link zum Kommentieren für Ihr Notebook mit uns 

---

**Anmerkungen**

- Wie schon in A2 verwenden wir tensorflow in der aktuellen Version 2. Solange diese Version nicht standardmäßig in Colab importiert wird, müssen wir dies explizit angeben:

In [ ]:
# Wir bestehen darauf, dass tensorflow 2 verwendet wird.
%tensorflow_version 2

- Auch für diese Aufgabensammlung sind wir wieder auf die Rechenpower von Google angewiesen und benötigen eine Laufzeitumgebung unter Verwendung einer GPU. Unter Umständen kann es bei der Bereitstellung zu Engpässen kommen. Stellen Sie also zunächst sicher, dass Ihnen eine GPU zugewiesen werden konnte (Sollte dies einmal nicht der Fall sein, bleibt Ihnen zumindest bei den darauf angewiesenen Teilen nichts anderen übrig als das Notebook zu einem späteren Zeitpunkt weiter zu bearbeiten.):

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available()

## 3.0 Visualisierung des bevorzugten Inputs
 
Wie Sie in der letzten Vorlesung gelernt haben, besteht ein neuronales Netz aus einer Ansammlung von *Zellen* (bei Keras *units* genannt), die miteinander verbunden sind. Jeder Input, der in das Netz gegeben wird, erzeugt in jeder Zelle eine *Aktivierung*, das heißt einen bestimmten Ausgabewert der Zelle. Die Zellen reagieren auf unterschiedliche Eingabedaten unterschiedlich stark, abhängig davon, welche Werte die Verbindungsgewichte während des Trainings angenommen haben. 
 
Auf welches Bild - oder allgemein welchen Input - eine Zelle besonders stark reagiert, gibt uns Aufschluss darüber, welche Informationen sie aus dem Input extrahiert und welche Rolle sie bei der Berechnung der Ausgabe des gesamten Netzwerks spielt. Wir können also etwas über die Funktionsweise eines Netzwerks lernen, wenn wir den *Input* so optimieren, dass der *Output* bestimmter Zellen maximiert wird.

### 3.0.0 Erzeugung eines trainierten Netzwerks
 
Um nach bevorzugten Inputs suchen zu können, benötigen wir zunächst ein Netzwerk, das bereits auf eine bestimmte Aufgabe trainiert wurde. Als anschauliches Beispiel benutzen wir hierfür das convolutional network `model3`, das wir im letzten Aufgabenblatt A2 erstellt haben, und trainieren es wieder auf den `MNIST`-Datensatz.

In den folgenden Code-Zellen werden die dafür nötigen Module importiert, der Datensatz heruntergeladen, das Netzwerk zusammengestellt und schließlich auf den Trainingsdaten trainiert.

---

**Anmerkungen**

Die genaue Anzahl der Filter und deren Größe kann von denen in A2 abweichen.

In [ ]:
# Import der neben tensorflow benötigten Module
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as display

In [ ]:
# Der MNIST-Datensatz wird heruntergeladen und skaliert.
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# wir skalieren unsere Grauwerte auf -1 bis 1 
train_images = (train_images/255 - 0.5) * 2
test_images = (test_images/255 -0.5) * 2

In [ ]:
# Erzeugung und Training eines convolutional networks wie in A2.
input_layer = tf.keras.layers.Conv2D(
    input_shape = (28, 28, 1),  # Input-Bildgröße und Anzahl der Farbkanäle
    filters = 5,                # wieviele verschiedene Filter verwendet werden
    kernel_size = (11, 11),      # Filtergröße
    activation= 'relu'
)

flatten_layer = tf.keras.layers.Flatten()

out_layer = tf.keras.layers.Dense( 
    units=10,                      # 10 units als Output
    activation='softmax',          # Die Softmax-Aktivierung zur Klassifikation 
    name = 'out'
)

model3 = tf.keras.models.Sequential([
    input_layer,
    flatten_layer,
    out_layer
])

model3.compile(
    loss = 'sparse_categorical_crossentropy',  # Klassifizierungsfehler
    metrics = ['accuracy'],                    
)

tf.keras.utils.plot_model(model3, show_shapes = True)

In [ ]:
model3.summary()

In [ ]:
model3.fit(
    x = train_images.reshape(-1, 28, 28, 1), 
    y = train_labels,
    batch_size=200,
    validation_data = (test_images.reshape(-1, 28, 28, 1), test_labels),
    epochs = 10   # Eine Epoche ist abgeschlossen wenn alle Datenpukte einmal trainiert wurden
)

### 3.0.1 Optimierung des Inputs

Das Ziel ist es nun, für dieses Netzwerk einen *Input* zu finden, der den *Ouput* einer bestimmten Zelle maximiert. Hierfür müssen wir uns eine Zelle aussuchen, deren Ausgabewert maximiert werden soll. Im diesem Fall ist es zunächst besonders anschaulich, eine Zelle aus der Ausgabeschicht zu wählen, da diese eine bestimmte Ziffer repräsentiert.

Die Methode `get_layer()` gibt uns Zugriff auf die einzelnen Schichten eines Modells. 

In [ ]:
inspection_layer = model3.get_layer('out')

Wir können nun auf den Ouput dieser Schicht zugreifen wie auf ein ``numpy ndarray`` und uns eine einzelne Zelle heraussuchen:

In [ ]:
inspection_unit = inspection_layer.output[:, 1]

Nun erzeugen wir ein neues Modell, das die gleichen Eingabeschichten besitzt, als Ausgabe jedoch nur die ausgewählte Zelle verwendet:

In [ ]:
unit_activation = tf.keras.Model(inputs = model3.input, outputs = inspection_unit)

In [ ]:
tf.keras.utils.plot_model(unit_activation)

In [ ]:
unit_activation.summary()

Wir haben nun also das Modell, dessen Output wir mittels eines geeigenten Inputs maximieren wollen. Hierfür erstellen wir eine Eingabevariable, die die passende Größe zu diesem Modell hat. Also letztlich ein Bild, das wir dem Netzwerk als Eingabe präsentieren können und im Rahmen der Optimierung verändern werden.

---
**Anmerkungen**

Diese Eingabevariable kann anfangs mit Zufallszahlen gefüllt sein - wir verwenden hier einheitlich 0.7 für jeden Bildpunkt, damit genügend Aktivität im Netz ausgelöst wird. Diese Mindestaktivität benötigen wir insbesondere bei den im späteren Verlauf der Übung verwendeten tiefen neuronalen Netzen, da wir ansonsten das Problem bekommen können, dass kleine Änderungen in der Eingabevariable zu kleine Auswirkungen auf die Aktivität tieferer Schichten zeigen.

In [ ]:
offset = .7
input_variable = tf.Variable(
    initial_value = tf.zeros(shape=(1, 28, 28, 1)) + offset,
    trainable=True
)

Unsere `input_variable` ist nun der einzige Teil, der in der anstehenden Optimierung verändert werden soll. Das Interface von Keras ist allerdings für das herkömmliche Trainieren von Netzwerken mit annotierten Daten ausgelegt, also das Optimieren von Gewichten. Daher brauchen wir an dieser Stelle etwas mehr Code, um Keras dazu zu bringen, den *Input* zu optimieren. Hierfür müssen wir unsere eigene *Trainingsschleife* schreiben. In jedem Durchlauf dieser Schleife wollen wir die folgenden Schritte durchführen:

1. **Die Berechnung der Ausgabe**: In jedem Schritt wird aus dem aktuellen Stand der Input-Variable die Aktivierung unserer Zielzelle berechnet.

2. **Die Berechnung des Fehlers**: Keras ist darauf ausgelegt, *Fehler* zu *minimieren*. In unserem Fall wollen wir aber die Aktivierung der Zelle *maximieren*. Dieses Ziel können wir erreichen, indem wir Keras den *negativen* Output minimieren lassen (also `loss = -output`). 

3. **Die Berechnung der Richtung**: Jeder Pixelwert beinflusst die Ausgabe des Netzwerks. Je nach Verschaltung der Gewichte kann der Output in Abhängigkeit vom Wert eines Pixels ab- oder zunehmen (oder sich auch so gut wie nicht verändern). Keras berechnet für jeden Pixel die Veränderungsrichtung, die den Fehler verkleinert. Mathematisch bezeichnet man diese Richtungen als *Gradienten*.  

4. **Die Veränderung der Input-Variable**: Jeder Pixel des Input-Bildes wird um einen kleinen Betrag in Richtung seines Gradienten verändert.

Diese Schritte werden in der Trainigsschleife so oft durchgeführt, bis die Abweichung vom gewünschten Ergebnis klein genug ist oder eine vorher festgelegte Anzahl an Interationen erreicht wurde. Im Folgenden sehen Sie eine mögliche Implementierung dieser Trainigsschleife mit Keras.

In [ ]:
# Wir wählen als Optimierungsalgorithmus den Adam-Algorithmus.
# Hierfür initialisieren ein entsprechendes Optimizer-Objekt.
# Dieses sammelt die Gradienten und steuert die Veränderungen der Input-Variable.
optimizer = tf.optimizers.Adam()
# Die Trainingsschleife beginnt. Es werden 500 Iterationen durchlaufen.
for i in range(500):
    # Es wird ein GradientTape geöffnet. Damit wird Tensorflow mitgeteilt,
    # dass für alle eingerückten Operationen die Gradienten aufgezeichnet 
    # werden sollen.
    with tf.GradientTape() as tape:
        # Die Aktivierung aus unserem Hilfsnetzwerk. 
        activation = unit_activation(input_variable)
        # Es wird gemittelt, falls mehrere units, ein Kanal oder eine ganze Schicht betrachtet werden sollen.
        output = tf.reduce_mean(activation)
        # Der Fehler ist die Abweichung vom bestmöglichen Output, also 1 - tatsächlicher Output.
        error = 1-output
    # Zur Kontrolle lassen wir alle 50 Iterationen den Fehler ausgeben.
    if i%20 ==0: print(f'Fehler: {error:.6f}')
    # Wenn die Abweichung klein genug ist, können wir abbrechen
    if error < 0.0000001:
        break
    # Andernfalls kann nach einem "besseren" Input gesucht werden:
    # Das GradientTape berechnet dafür, in welche Richtung (Gradient) die
    # Variablen verändert werden müssen, um den Fehler zu verkleinern
    gradients = tape.gradient(error, input_variable)       
    # Der Optimizer berechnet nun aus den Gradienten ein Update für jeden Pixel     
    optimizer.apply_gradients([(gradients, input_variable)])     
    # Wir beschränken die Lösung auf Werte zwischen -1 und 1, um den Wertebereich
    # der Trainingsdaten nicht zu verlassen.
    input_variable.assign(tf.clip_by_value(input_variable, -1, 1))
    

Nach jeder Iteration nimmt der Fehler ab, d.h. die Ausgabe der ausgewählten Zelle nähert sich 0. Sind wir nahe genug, können uns nun das optimierte Inputbild anzeigen lassen.

In [ ]:
plt.imshow(np.reshape(input_variable.value(), (28,28)), cmap=plt.cm.binary_r)
_ = plt.colorbar()

In dem entstandenen Bild können wir mit etwas gutem Willen die Umrisse der zugehörigen Ziffer erkennen. Es ist eine Art "Mutter aller Dreien", eine Vorstellung von den Mustern im Eingaberaum, die das Netzwerk während des Trainings bezogen auf die Kategorie $3$ entwickelt hat.

### 3.0.2 Aufgabe: Alternative Zahlen

Verändern Sie den Code so, dass die Aktivierung für eine andere Zahl optimiert wird, und stellen Sie das entstandene Bild des bevorzugten Inputs dar. 

## 3.1 Merkmalsvisualisierung in großen Netzen

Wir haben zuvor am Beispiel eines einfachen Netzwerks gelernt, wie wir die bevorzugten Merkmale einzelner Zellen visualisieren können. Wir können diese Techniken ebenso auf große, vortrainierte Netzwerke aus `keras.applications` anwenden, um einen Eindruck davon zu bekommen, wie die Informationsverarbeitung in tiefen Netzen vor sich geht, die auf Unmengen von Bildern trainiert wurden.

### 3.1.0 Laden des Netzwerks

Wir laden diesmal das Netz `InceptionV3` herunter, ein guter Kompromiss zwischen Performance und Rechenaufwand:

In [ ]:
model = tf.keras.applications.InceptionV3(include_top=False)

In [ ]:
tf.keras.utils.plot_model(model, dpi=20)

In [ ]:
model.summary()

Wie Sie sehen, handelt es sich hierbei wieder um ein riesiges Netzwerk mit fast 22 Millionen Parametern.

Die Visualisierung der bevorzugten Merkmale einzelner Zellen oder Schichten funktioniert im Prinzip genauso wie bei dem einfachen Netzwerk, das wir zuvor selbst erstellt haben. Wir wählen eine Zelle aus und optimieren dann das Inputbild so, dass die Aktivierung maximiert wird. Dieses Prinzip kann auch auf ganze Schichten angewendet werden. Hierfür maximieren wir einfach die mittlere Aktivierung der gewählten Schicht.

### 3.1.0 Definition der Optimierungsfunktion 

Da wir die Operation mehrfach für verschiedene Fälle anwenden wollen, bietet es sich an, den entsprechenden Code in einer Funktion zu kapseln. Da wir die Methode in diesem Fall auch auf große Netzwerke anwenden wollen, bauen wir in die Funktion einen zusätzlichen Kniff ein: Das Inputbild wird immer wieder zufällig leicht verschoben. Dies macht die Optimierung stabiler und führt zu glatteren Ergebnisbildern. 

Den Code zum verschieben der Bidler müssen Sie nicht im Einzelnen verstehen. In der Funktion `maximize_activation()` können Sie die einzelnen Schritte wiedererkennen, die wir bereits weiter oben verwendet haben.

In [ ]:
#@title 
#@markdown Bitte ausführen: Code zum zufälligen Verschieben eines Bildes.

def random_shift(img, maxroll):
  # Randomly shift the image to avoid tiled boundaries.
  if maxroll > 0:
    shift = tf.random.uniform(shape=[2], minval=-maxroll, maxval=maxroll, dtype=tf.int32)
    shift_down, shift_right = shift[0], shift[1] 
    img_shifted = tf.roll(tf.roll(img, shift_right, axis=1), shift_down, axis=0)
  else:
    img_shifted = img
  return img_shifted

In [ ]:
# Wir maximieren wieder genau wie oben, jedoch "wackeln" wir mit dem Bild, das
# wir optimieren, in jedem Schritt ein Stück hin- und her.
# Das führt dazu, dass in unserer Lösung Pixel-Werte nicht allzu stark springen. 

def maximize_activation(input_variable, activation, steps, max_shift):
    """ Function that optimizes the 'input_variable' to maximize the output of
        'activation'.

        input_variable:  a tf.variable with shape as expected by 'unit_activation'
        activation:      a function that produces an activation value from 'input_variable'
        steps:           number of iterations
        max_shift:       maximum number of pixels the image is randomly shifted by.

        returns: the optimized inut_variable 
        """
    optimizer = tf.optimizers.Adam() #learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
    for i in range(steps):  
        with tf.GradientTape() as tape:
            x_shift = random_shift(input_variable, max_shift)          # Wir schieben unser Bild immer zufällig etwas zur Seite
            shifted_activation = activation(x_shift)
            error = -tf.reduce_mean(shifted_activation)
        if i%100 == 0: 
            print('Fehler: %.2f'%(error.numpy()))
        gradients = tape.gradient(error, input_variable)               # Der Gradient gibt an, wie ich eine Variable (hier input_variable) verbessern muss, um den Fehler zu reduzieren
        optimizer.apply_gradients([(gradients, input_variable)])       # Der optimizer verbessert input_variable in Richtung des Gradienten
       
        input_variable.assign(tf.clip_by_value(input_variable, -1, 1)) # Hier beschränken wir unsere Lösung auf Werte zwischen -1 und 1 wie in einem "richtigem" Bild
    
    return input_variable

### 3.1.1 Optimierung einzelner Filter

 Wie zuvor suchen wir uns eine Zelle aus einer Schicht aus dem Netzwerk aus, für die wir das Eingabebild optimieren wollen. Diesmal nehmen wir eine der früheren Schichten, um die grundlegenden Merkmale zu untersuchen, mit denen das Netzwerk arbeitet.

In [ ]:
inspection_layer = model.get_layer('mixed1')
inspection_unit = inspection_layer.output[:, 9, 9, 30] # irgendeine Zelle aus der Schicht
unit_activation = tf.keras.Model(inputs = model.input, outputs = inspection_unit)

Wir definieren nun wieder eine Inputvariable und wenden unsere Funktion darauf an:

In [ ]:
offset = 0.
input_variable = tf.Variable(
    initial_value = tf.zeros(shape=(1, 200, 200, 3)) + offset,
    trainable=True
)
input_variable = maximize_activation(input_variable, unit_activation,steps=2000,max_shift=4)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow((np.squeeze(input_variable.value())+1)/2)

Wir sehen, dass die von uns gewählte Zelle auf bestimmte Strukturen im Input besonders stark reagiert. In den niedrigeren Schichten der Bilderkennungsnetze reagieren die Zellen, ähnlich wie in biologischen visuellen Systemen, auf einfache Merkmale wie Kanten und Linien in bestimmten Ausrichtungen. 

Es fällt auf, dass das Bild nur in einem kleinen Bereich Strukturen aufweist, während der Rest vom Optimierungsalgorithmus nicht verändert wurde. Das liegt daran, dass wir eine einzelne *Position* dieses Filters aus dem convolutional network betrachtet haben und der Input für alle Positionen außerhalb des durch diese Filterposition beschriebenen "rezeptiven Feldes" die Aktivierung der gewählten Zelle beeinflussen und somit auch keine Gradienten aufweisen.

### 3.1.2 Maximierung ganzer Kanäle 

Unsere Funktion für die Maximierung der Aktivierung haben wir so geschrieben, dass sie auch mit mehreren Zellen gleichzeitig zurechtkommt. Anstatt einer einzelnen Position eines Filters können wir also auch den ganzen Kanal maximieren, also sämtliche Positionen eines bestimmten Filters. Hierfür müssen wir lediglich in unserer Schicht anstelle der konkreten Position (`[:,9,9,30]` - Pixelposition 9,9 des 30. Filters) den ganzen Bereich auswählen (`[:,:,:,30]` - alle Positionen des 30. Filters).

Wir definieren also mit dem neuen Bereich unserer `inspection_layer` eine neue `channel_activation`-Funktion:

In [ ]:
inspection_channel = inspection_layer.output[:, :, :, 30] # ganzer Kanal, vergleichen Sie mit dem Index weiter oben
channel_activation = tf.keras.Model(inputs = model.input, outputs = inspection_channel)

Und wenden wie zuvor unsere Funktion darauf an:

In [ ]:
offset = 0.
input_variable = tf.Variable(
    initial_value = tf.zeros(shape=(1, 200, 200, 3)) + offset,
    trainable=True
)
input_variable = maximize_activation(input_variable, channel_activation, 2000, 4)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow((np.squeeze(input_variable.value())+1)/2)

Das optimale Bild besteht nun aus einer sich wiederholenden Struktur, deren Elemente dem Muster des vorherigen Bildes entsprechen.

### 3.1.3 Aufgabe: Visualisierung anderer Kanäle der gleichen Schicht

Visualisieren Sie den optimalen Input für einen oder mehrere andere Kanäle der `mixed1`-Schicht.

In [ ]:
# Lösung:

### 3.1.4 Aufgabe: Visualisierung der Kanäle einer tieferen Schicht

Visualisieren Sie den optimalen Input für einen oder mehrere Kanäle aus einer tieferen Schicht (mixed2, mixed3, ... mixed10) 

---
**Anmerkungen**

Achtung: Je tiefer die Schicht, desto schwieriger wird die Optimierung! Sollte die Optimierung keine zufriedenstellenden Ergebnisse liefern, können Sie versuchen, einen höheren `offset` zu verwenden.


In [ ]:
# Lösung:

## 3.2 Deep Dream

2015 veröffentlichte Google einige Artikel mit Bildern, die als die *Träume* tiefer Netze bezeichnet wurden. Wie Sie sich mittlerweile vielleicht denken können, beruhen diese Bilder auf ähnlichen Techniken wie die, die Sie gerade kennegelernt haben. Anstelle eines leeren Bildes, wird die Optimierung hier allerdings mit einem beliebigen Foto begonnen. Die Aktivität des Netzwerks wird also in eine bestimmte Richtung geleitet. Aus dieser Augangsposition erzeugt man durch das Maximieren der Aktivität bestimmter Schichten also so etwas wie Assoziationen, die das Netzwerk gelernt hat.

### 3.2.0 Inputbild

Wir laden also zunächst ein Bild herunter, wie Sie es im zweiten Aufgabenblatt gelernt haben:

In [ ]:
!wget -O wolken.jpg https://upload.wikimedia.org/wikipedia/commons/thumb/b/bb/Wolken_%C3%BCber_Boksee.jpg/256px-Wolken_%C3%BCber_Boksee.jpg
wolken = plt.imread('wolken.jpg')
plt.figure(figsize=(10,10))
plt.imshow(wolken)

Damit das `Inception`-Netzwerk das Bild interpretieren kann, muss es noch durch das zugehörige Preprocessing geschoben werden.

In [ ]:
wolken_pp = tf.keras.applications.inception_v3.preprocess_input(wolken)

### 3.2.0 Definition der Optimierungsschicht

Im Unterschied zu den vorherigen Abschnitten wird diesmal eine ganze Schicht des Netzwerks maximiert. Ansonsten ist der Code für die Aktivierungsfunktion wie gehabt:

In [ ]:
inspection_layer = model.get_layer('mixed3')
layer_activation = tf.keras.Model(inputs = model.input, outputs = inspection_layer.output)

Unsere `input_variable` wird dieses Mal mit dem Bild initialisiert:

In [ ]:
input_variable = tf.Variable(
    initial_value = wolken_pp[np.newaxis],
    trainable=True
)
input_variable.shape

### 3.2.1 Optimierung

Bei der Optimierung wird ein zusätzlicher Trick angewandt: Nach einigen Iterationen wird das Bild jeweils vergrößert. Das hat zur Folge, dass das Netzwerk Details auf unterschiedlichen Größenskalen in das Bild hineinassoziert, und wurde aus rein ästhetischen Gründen so gemacht.

Im Code haben wir daher eine zusätzliche Schleife über die Anzahl der vorgenommenen Vergrößerungen. In jedem Durchlauf wird unsere Funktion zur Maximierung der Aktivierung ausgeführt.

In [ ]:
zoom = 1.3 # Vergrößerungsfaktor
zoom_steps = 3 # wie oft vergrößert werden soll

for n in range(zoom_steps):
    # In jedem zoom_step vergrößern wir unser Bild
    print('=== Zoom-Schritt: ', n, '===')
    if n > 0: # zoom into image
        old_shape = np.array(input_variable.numpy().shape[1:3])  
        new_shape = tf.cast(old_shape*zoom, tf.int32)
        input_variable = tf.Variable(initial_value = tf.image.resize(input_variable, new_shape))

    input_variable = maximize_activation(input_variable, layer_activation, 1000, 100)

    display.clear_output(wait=True)
    display.display(tf.keras.preprocessing.image.array_to_img((input_variable[0]+1)/2))

### 3.2.2 Aufgabe: Vergrößerungsfaktor und Zoom-Schritte

Welchen Einfluss haben Vergrößerungsfaktor und Anzahl der Zoom-Schritte auf das resultierende Bild? Verändern Sie hierfür die Anzahl der Zoomschritte und den Vergrößerungsfaktor. Probieren Sie einige Kombinationen aus. Was geschieht für viele Schritte mit kleinem Faktor, für wenige Schritte mit großem Faktor?

In [ ]:
# Lösung:

### 3.2.3 Aufgabe: Eigenes Bild verwenden

Laden Sie ein anderes Bild Ihrer Wahl herunter und wenden Sie das Deep-Dreaming darauf an. Experimentieren Sie auch mit anderen Schichten im Netzwerk für die Optimierung.

In [ ]:
# Lösung:

## 3.3 Adversarial Training

In den vorangegangenen Abschnitten haben wir gelernt, wie wir ein neuronales Netzwerk auf eine Art und Weise benutzen können, für die es ursprünglich nicht gedacht war. Wir gehen jetzt noch einen Schritt weiter und starten einen *Angriff* auf unser Netzwerk mit Hilfe des sogenannten *Adversarial Training* (*Gegnerisches Trainieren*). Hierbei werden gezielt Inputs generiert, die ein bestimmtes Netzwerk aus dem Konzept bringen sollen. 

Wir können ein Bild so verändern, dass es von einem Bilderkennungsnetzwerk nicht mehr erkannt wird, obwohl für das menschliche Auge kaum ein Unterschied zu sehen ist. Hierfür benutzen wir ganz ähnliche Techniken wie zuvor.

### 3.3.0 Ungehinderte Klassifikation

Wir wollen zunächst wieder ein vortrainiertes Netzwerk herunterladen und es auf ein Bild anwenden. Diesmal nehmen wir das `MobileNetV2`.

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions

model = MobileNetV2(include_top=True, weights='imagenet')

Außerdem benötigen wir natürlich wieder ein Bild, das wir klassifizieren wollen.

In [ ]:
!wget -O example.jpg https://upload.wikimedia.org/wikipedia/commons/thumb/7/7a/Grand_Ducal_Police_car_%28Ford%29_in_Luxembourg_City.jpg/320px-Grand_Ducal_Police_car_%28Ford%29_in_Luxembourg_City.jpg

Das Bild wird wie immer zunächst dem netzwerkspezifischen Preprocessing unterzogen.

In [ ]:
img = tf.keras.preprocessing.image.load_img('./example.jpg', target_size=(224,224))

img_pp = preprocess_input(tf.keras.preprocessing.image.img_to_array(img))
img_pp = img_pp[None, ...]

img_pp.shape

In [ ]:
fig = plt.figure(figsize=(7,7))
plt.imshow(img_pp.squeeze())

In [ ]:
prediction = model.predict(img_pp)
decode_predictions(prediction, top=3)

Das Netzwerk erkennt richtig, dass es sich um ein Polizeifahrzeug handelt.

### 3.3.1 Störung der Klassifikation

Zuvor haben wir ein Inputbild dahingehend verändern wollen, dass die Aktivierung einer bestimmten Zelle maximiert wird. Umgekehrt ist es natürlich auch möglich, die Aktivierung einer bestimmten Zelle zu *minimieren*. Bei der Klassifikation *gewinnt* die Zelle mit dem höchsten Output. In diesem Fall war es die Zelle, die beim Training dem Begriff `police_van` zugeordnet war. 

Wollen wir die Klassifikation stören, so können wir mit unserer Methode also die Richtung ermitteln, in die jeder Bildpunkt verändert werden muss, um den Output der `police_van`-Zelle zu reduzieren.

In [ ]:
def get_gradient_direction(model_input, label):

    loss_function = tf.keras.losses.SparseCategoricalCrossentropy()

    with tf.GradientTape() as tape:
        model_output = model(model_input)
        loss = loss_function(label, model_output)

    # Get the gradients of the loss w.r.t to the input image.
    gradient = tape.gradient(loss, model_input)
    # Get the sign of the gradients to create the perturbation
    gradient_direction = tf.sign(gradient)

    return gradient_direction

Wir wenden diese Funktion nun auf die ``police_van``-Zelle an. Als Anfangswert wird der Variablen unser Polizeiautobild gegeben. Sie errechnet uns dann für jeden Pixel eine Richung und gibt diese zurück.

In [ ]:
label_id = 734 #police_van_index (https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a)

x = tf.Variable(
    initial_value = img_pp,
    trainable=True
)

gradient_direction = get_gradient_direction(x, label_id)

Diese Richtungen könne wir als Bild darstellen lassen. Für das menschliche Auge ist hier keine offensichtliche Struktur erkennbar.

In [ ]:
plt.imshow(gradient_direction.numpy().squeeze())

Wir können nun diese Optimierungsrichtung auf unser ursprüngliches Bild addieren. Hierbei verwenden wir einen kleinen Faktor, so dass das Ausgangsbild möglichst wenig gestört wird. 

In [ ]:
adversarial_img = (img_pp + 0.005 *gradient_direction.numpy())

In [ ]:
fig = plt.figure(figsize=(7,7))
plt.imshow(adversarial_img.squeeze())

Unser Bild sieht quasi unverändert aus.

In [ ]:
prediction = model.predict(adversarial_img)
decode_predictions(prediction, top=3)

Das Netzwerk erkennt es jedoch jetzt nicht mehr als Polizeifahrzeug sondern meint ein Rennauto vor sich zu haben. 

### 3.3.2 Aufgabe: Weitere Störung

Suchen Sie sich auf [github](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a) den Index für die Klasse `racer` und stören Sie auch diese Klasse im `adversarial_img`. Was passiert mit dem Klassifizierungsergebnis?

In [ ]:
# Lösung:

### 3.3.4 Aufgabe: Eigenes Beispiel

Laden Sie ein anderes Bild Ihrer Wahl herunter und stören Sie die Klassifikation. Für der Auswahl eines geeigneten Bildes können Sie sich von den unterschiedlichen Klassen, auf die das Netzwerk trainiert wurde, inspirieren lassen.

In [ ]:
# Lösung:

## 3.4 OPTIONAL: Image style transfer revisited

Im Notebook A1 haben Sie bereits den Image Style Transfer bzw. konkreter den Neural Image Style Transfer (NIST) kennengelernt. Dabei benutzten wir eine Funktion, die aus einem *Style*- und einem *Contentbild* ein neues Bild erzeugte, ohne uns damit auseinanderzusetzen, was im Inneren der Funktion geschieht. Wir werden nun sehen wie mit den Techniken, die Sie im Rahmen dieses Notebooks kennengelernt haben, auch der NIST umgesetzt werden kann.

Die konkrete Implementierung des NIST ist relativ komplex. Dieser Teil des Notebooks ist daher optional. Machen Sie sich also keine Sorgen, wenn Sie hier nicht ganz durchblicken oder wenn Ihnen der bisherige Stoff schon genug zu Schaffen gemacht hat. 

Wir beginnen wieder damit, uns jeweils ein Bild für den *Style* und den *Content* herunterzuladen.

In [ ]:
content_path = tf.keras.utils.get_file('content.png', 'https://www.stnds.de/damfiles/article_img_12/was-wir-foerdern/programme/link/LINK_Logo_WEB_RGB.png_200-1e31a9f7a822254b191526ea16cd0c79.png')
style_path = tf.keras.utils.get_file('style.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg')

In [ ]:
content_img = tf.keras.preprocessing.image.load_img(content_path)
style_img = tf.keras.preprocessing.image.load_img(style_path)

plt.figure(figsize=(15,8))

plt.subplot(1, 2, 1)
plt.imshow(content_img)

plt.subplot(1, 2, 2)
plt.imshow(style_img)

Der NIST beruht auf der Beobachtung, dass in einem ganz bestimmten vortrainierten Netzwerk, dem `VGG19`, in ganz bestimmten Schichten der Stil eines Bildes kodiert ist, während der Bildinhalt (da dieser noch abstrakter ist) in einer anderen, höheren Schicht abgebildet wird. 

Die Autoren der ursprünglichen [Studie](https://arxiv.org/abs/1508.06576) kamen daraufhin auf die Idee, dass man ein Bild so optimieren kann, dass es in den Style-Schichten eine ähnliche Aktivität erzeugt wie *ein* Bild, während in der Content-Schicht die Aktivierung eines *anderen* Bildes nachgeahmt wird.   

Die Funktionsweise des NIST ist also in gewisser Weise ein Artefakt genau dieses Netzwerks und konnte bisher nicht in dieser Form in anderen Netzen wiederholt werden.

Wir laden uns zunächst das Netz `VGG19` herunter. Dieses ist wie gewohnt in `tf.keras.applications` zu finden. Wir verwenden eine vortrainierte Instanz des Netzwerks, indem wir den Parameter `weights` entsprechend spezifizieren.

In [ ]:
from tensorflow.keras.applications import vgg19
model = vgg19.VGG19(include_top=False, weights='imagenet')

In [ ]:
model.summary()

Wie wir sehen hat das Modell etwas mehr als 20 Millionen Parameter. Diese wollen wir aber nicht trainieren - hier wird wieder lediglich der *Input* optimiert.

Der [Publikation](https://arxiv.org/abs/1508.06576) können wir die Schichten des Netzwerkes entnehmen, in der *Style* und *Content* repräsentiert sind.

In [ ]:
# Content layer where will pull our feature maps
content_layers = ['block5_conv2'] 

# Style layer of interest
style_layers = [
    'block1_conv1',
    'block2_conv1',
    'block3_conv1', 
    'block4_conv1', 
    'block5_conv1'
]

Wir benötigen nur diesen Teil des Netzwerkes. Wir können uns also ein neues Modell erzeugen, das uns genau die gewünschten Schichtaktivitäten als Output gibt.


In [ ]:
layer_outputs = [model.get_layer(name).output for name in style_layers + content_layers]
model = tf.keras.Model([model.input], layer_outputs)

Im Folgenden schreiben wir eine Funktion, die aus der Ausgabe des Modells die *Style*- und *Content*-Komponenten extrahiert. Leider ist es nicht ganz so einfach, eine geeignete aktivitätsabhängige Repräsentation des Bildstils zu erhalten. Hierfür wird anhand der Aktivität der Styleschichten die sogenannte [Gramsche Matrix](https://de.wikipedia.org/wiki/Gramsche_Determinante) berechnet, welche die Korrelationen zwischen den einzelnen Filtern misst. Hierüber müssen Sie sich im Detail keine Gedanken machen...

In [ ]:
#@title 
#@markdown Bitte ausführen: Code zum Extrahieren der Stil- und Inhaltsmerkmale eines Bildes.
# Der Stil eines Bildes in Faltungsnetzen kann durch die Korrelation der 
# Filter beschrieben werden.

# Funktion zum Berechnen der Korrelation zwischen einzelnen Filtern in einem Array
def gram_matrix(array):

  result = tf.linalg.einsum('bijc,bijd->bcd', array, array)
  input_shape = tf.shape(array)
  num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
  
  return result/(num_locations)

# Funktion zum Extrahieren der Style- und Contentbeschreibung aus dem Output 
def seperate_style_and_content(outputs):

    style_outputs  = outputs[:len(style_layers)]
    style_outputs = [gram_matrix(style_output) for style_output in style_outputs]

    content_outputs = outputs[len(style_layers):]

    return style_outputs, content_outputs



Wie Sie es schon zuvor gesehen haben, müssen wir die Bilder noch einer netzwerkspezifischen Vorverarbeitung unterziehen. Anschließend können wir die Zielvorgaben der *Style*- und *Content*merkmale anhand unserer beiden Bilder errechnen lassen.

In [ ]:
max_size = 512 # setzt die Größe fest, auf die wir unser Bild skalieren

# Input Vorbereitung für das Style Image
# 1. Bild in Array umwandeln
style_img_array = tf.keras.preprocessing.image.img_to_array(style_img)
# 2. Bild auf maximale Größe skalieren
style_img_array = tf.image.resize(style_img_array, (max_size, max_size), preserve_aspect_ratio=True)
# 3. Gleiche Vorverarbeitung anwenden wie im Training
style_img_prepro = vgg19.preprocess_input(style_img_array)
# 4. Eine neue Dimension hinzufügen, damit das Bild die Standardform (1, Höhe, Breite, Farbkanäle) hat
style_img_prepro = style_img_prepro[tf.newaxis, :]

style_image_output = model(style_img_prepro)
style_image_style, style_image_content = seperate_style_and_content(style_image_output)

Das gleiche setzen wir jetzt für das Contentbild um:

In [ ]:
content_img_array = tf.keras.preprocessing.image.img_to_array(content_img)
content_img_array = tf.image.resize(content_img_array, (max_size, max_size), preserve_aspect_ratio=True)
content_img_prepro = vgg19.preprocess_input(content_img_array)
content_img_prepro = content_img_prepro[tf.newaxis, :]

content_image_output = model(content_img_prepro)
content_image_style, content_image_content = seperate_style_and_content(content_image_output)

An diesem Punkt stehen nun zwei Zielvorgaben für den NIST fest: In der Variable `style_image_style` sind die stilrepräsentierenden (verrechneten) sowie in `content_image_content` die den grundsätzlichen Bildinhalt repräsentierden Aktivitätsmuster gespeichert, die es nun gleichzeitig mit einem zu optimierenden Input zu erreichen gilt.

### 3.4.0 Definition des Fehlers 

Unser Ziel ist also ein Bild, das den *Content* des Content-Bildes wiederspiegelt und den *Style* des Style-Bildes (also die entsprechenden Aktivitätsmuster in Netzwerk auslöst). Der Fehler setzt sich somit zusammen aus den Abweichungen
- zwischen den Style-Merkmalen von Ziel-Bild und Style-Bild sowie
- zwischen den Content-Merkmalen von Ziel-Bild und Content-Bild.

In [ ]:
#@title
#@markdown Bitte ausführen: Definition des Fehlers

style_weight = 1e-2
content_weight = 1e4

def style_content_loss(outputs):
    
    style_output, content_output = seperate_style_and_content(outputs)
    
    style_loss = tf.add_n(
        [tf.reduce_mean((style_output[i] - style_image_style[i])**2) 
         for i in range(len(style_layers))]
    )
    style_loss *= style_weight / len(style_layers)

    content_loss = tf.add_n(
        [tf.reduce_mean((content_output[i] - content_image_content[i])**2) 
         for i in range(len(content_layers))]
    )
    content_loss *= content_weight / len(content_layers)
    
    loss = style_loss + content_loss
    
    return loss

### 3.4.1 Optimierung

Jetzt müssen wir wieder einen Optimierer sowie die Startversion des zu optimierenden Eingabebildes initialisieren.

In [ ]:
optimizer = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
input_variable = tf.Variable(
    initial_value = content_img_array[tf.newaxis, :] / 255,    # wir initalisieren unsere Variable mit dem Content-Bild
    trainable=True
)

Schließlich fehlt uns noch die zugehörige Optimierungsschleife. Wir können das Eingabebild schrittweise optimieren, indem wir dessen Content- und Style-Abstände zu den enstprechenden Bildern verkleinern.

In [ ]:
for i in range(300):  
    with tf.GradientTape() as tape:
        prepro_input = input_variable * 255
        prepro_input = vgg19.preprocess_input(prepro_input)
        output = model(prepro_input)
        loss = style_content_loss(output)
    print('.', end='')
    gradients = tape.gradient(loss, input_variable)            # der Gradient gibt an, wie ich eine Variable (hier x) verändern muss, um den Fehler zu verkleinern
    optimizer.apply_gradients([(gradients, input_variable)])   # der Optimizer verändert x in Richtung des Gradienten
    input_variable.assign(tf.clip_by_value(input_variable, 0, 1))  
    # Zwischenergebnis anzeigen
    if i%100 == 5:
        display.clear_output(wait=True)
        display.display(tf.keras.preprocessing.image.array_to_img(input_variable[0]))
# Endergebnis anzeigen
display.clear_output(wait=True)
display.display(tf.keras.preprocessing.image.array_to_img(input_variable[0]))

### 3.4.2 NIST mit eigenen Bildern

Jetzt können Sie die hier vorgestelle Impelmentierung der Stilübertragung für Paarungen selbst ausgesuchter Bilder anwenden, indem Sie für diese die Vorgaben bzgl. Style- und Contentbild berechnen lassen und den Input (mit dem Contentbild initialisiert) entsprechend optimieren lassen.